In [2]:
import streamlit as st
import pandas as pd
import altair as alt
import numpy as np
from utils import get_course_df
from utils import get_instructor_df
from utils import get_dept_df
from utils import options_map_to_columns

In [3]:
numeric_columns = [
        'hrs_per_week',
        'total_students',
        'responses',
        'response_rate',
        'interest_in_student_learning',
        'clearly_explain_course_requirements',
        'clear_learning_objectives_and_goals',
        'instructor_provides_feedback',
        'demonstrate_importance_of_subject_matter',
        'explains_subject_matter_of_course',
        'show_respect_for_all_students',
        'overall_teaching_rate',
        'overall_course_rate'
    ]

In [14]:
def load_data():
    # Load the yelp data.
    fce_data_url = "../fce-data.csv"
    raw_df = pd.read_csv(fce_data_url)
    df = raw_df
    df.dropna(inplace=True)
    # Convert numeric columns
    for nc in numeric_columns:
        df[nc] = pd.to_numeric(df[nc])

    # Filter data
    df = df[(df['total_students'] > 5) & (df['total_students'] <= 325)] # Remove Outlier
    df = df[df['response_rate'] > 0.4]
    df = df[df['college'] != 'Teaching Assistants']

    # df['overall_rate'] = df[(df['overall_course_rate'] + df['overall_teaching_rate'] + df['show_respect_for_all_students']) / 3]
    return df

In [16]:
df = load_data()
course_df = get_course_df(df)

options_map_to_column = options_map_to_columns

In [17]:
options_columns = ['overall_teaching_rate', 'overall_course_rate']
course_options = ['2021 Fall 85738 ED GOALS INST ASSESS', '2021 Spring 5840 TOOLS ONLINE LRNG']
course_df["course_name_code_year"] = course_df["year"].astype(str) + ' ' + course_df["semester"] + ' '  + course_df["num"].astype(str) + ' ' + course_df["course_name"]


In [18]:
filter_df1 = course_df.groupby(['course_name_code_year', 'instructor', 'responses'])[options_columns].mean().reset_index()
# st.write(filter_df1)
filter_df = filter_df1.loc[course_df['course_name_code_year'].isin(course_options)]
# st.write(filter_df)
reshaped_filter_df = filter_df.melt(id_vars=['course_name_code_year', 'instructor', 'responses'], var_name = 'Judge Parameter', value_name = 'Rating').sort_values(by='course_name_code_year').reset_index(drop=True)
# st.write(reshaped_filter_df)
selection_legend = alt.selection_multi(fields=['course_name_code_year'], bind='legend')

base = alt.Chart(reshaped_filter_df).mark_bar(tooltip=True).transform_calculate(
    y="split(datum.y, '_')"
).encode(
    y=alt.Y('course_name_code_year:O', type="nominal", axis=alt.Axis(title=None, labels=False)),
    x=alt.X('Rating:Q',axis=alt.Axis(grid=False)),
    color=alt.Color('course_name_code_year:N',legend=alt.Legend(title="Course Name", labelFontSize=12)),
    opacity=alt.condition(selection_legend, alt.value(1), alt.value(0.2)),
    row=alt.Row('Judge Parameter:N', header=alt.Header(labelAngle=0, labelAlign="left", labelFontSize=8)),
    tooltip = [alt.Tooltip(field = "course_name_code_year", title = "Course Name", type = "nominal"),
            alt.Tooltip(field = "Rating", title ="Rating", type = "quantitative", format=".2f"),
            alt.Tooltip(field = "instructor", title ="Instructor", type = "nominal"),
            alt.Tooltip(field = "responses", title ="Responses", type = "quantitative")
    ],
).properties(
    title="Compare Ratings for Selected Courses", 
    width=400,
).configure_title(
    anchor='middle',
).add_selection(
    selection_legend
).configure_legend(
    labelLimit= 0
)


In [19]:
base

alt.Chart(...)